# Q2: Simulating Attitude Dynamics (50 pts)
We're going to simulate the attitude dynamics of a rigid body like this T-wrench: https://www.youtube.com/watch?v=1n-HMSCDYtM

In [2]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/devel/dynamics-simulation-16-715/HW1_solutions/Project.toml`


In [3]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Test
using Random
using Plots
plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /Users/kevintracy/.julia/packages/Plots/Di0Qb/src/backends.jl:435


Plots.PlotlyBackend()

# Part (a): Rotation functions (5 pts)

In [41]:
#Some standard functions for dealing with rotation matrices and quaternions from the class notes
function hat(ω)
    # TODO: implement the hat function 
    ω̂ = I(3)
    
    return ω̂
end

function L(q)
    # TODO: implement L 
    _L = zeros(4,4)
    
    return _L
end

function R(q)
    # TODO: implement R 
    _R = zeros(4,4)
    
    return _R
end

# TODO: implement H function 
H = zeros(4,3)


# add tests here 
@testset "rotation functions" begin 
    x = randn(3)
    y = randn(3)
    # test hat 
    @test isapprox(cross(x,y),hat(x)*y,atol=1e-12)
    @test isapprox(cross(y,x),hat(y)*x,atol=1e-12)
    @test isapprox(tr(hat(x)),0,atol=1e-12)
    @test isapprox(tr(hat(y)),0,atol=1e-12)
    @test isapprox(norm(hat(x) + hat(x)'),0,atol=1e-12)
    
    # test orthogonality of L and R 
    L1 = L(normalize(randn(4)))
    R1 = L(normalize(randn(4)))
    @test isapprox(norm(L1*L1'-I),0,atol=1e-10)
    @test isapprox(norm(R1*R1'-I),0,atol=1e-10)

    # make sure the rotation is correct, and H does what it's supposed to
    Random.seed!(1234)
    r = normalize(randn(3))
    θ = 0.3 
    ϕ = r*θ
    q = [cos(θ/2);r*sin(θ/2)]
    Q = exp(hat(ϕ))
    x = randn(3)
    x2 = Q*x
    # this tests H 
    x3 = H'*L(q)*R(q)'*H*x
    @test isapprox(x2,x3,atol = 1e-10)
end

Test Summary:      | Pass  Total
rotation functions |    8      8


Test.DefaultTestSet("rotation functions", Any[], 8, false, false)

# Part (b): Attitude Dynamics (25 pts)

In [42]:
#Inertia matrix we'll be using
J = Diagonal([1.0; 2.0; 3.0])

3×3 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅ 
  ⋅   2.0   ⋅ 
  ⋅    ⋅   3.0

In [43]:
#Implement Euler's equation for a rigid body
function dynamics_ω(ω)
    
    # TODO: add Euler's equation 
    ω̇ = zeros(length(ω))
    
    return ω̇
end

@testset "Euler's" begin
    ω1 = [.2;7;.2]
    α1 = [-1.4;0.04;-0.46666666666666]
    @test isapprox(α1, dynamics_ω(ω1),atol=1e-10)
end

Test Summary: | Pass  Total
Euler's       |    1      1


Test.DefaultTestSet("Euler's", Any[], 1, false, false)

In [44]:
#Implement the full attitude dynamics using a rotation matrix
function dynamics_Q(x)
    # input: x = [vec(Q);ω]
    
    # TODO: implement 
    ẋ = zeros(length(x))
    
    return ẋ
end

function dynamics_Q!(ẋ,x,p,t)
    # make the ODE in-place so it works with OrdinaryDiffEq
    ẋ .= dynamics_Q(x)
end

@testset "attitude dynamics (DCM)" begin 
    Q1 = exp(hat([1;2;3]))
    ω1 = [0.2;0.3;0.7]
    x1 = [vec(Q1);ω1]
    ẋ1 = [0.47267683571087754,-0.4926072371847232,0.337512546219523,
          0.5043029621213005,0.3210433517191289, -0.41546322185318607,
         -0.35118036539795094,0.003154917030294274,0.08162351044578747,
         -0.2099999999999999,0.14,-0.02]
    
    @test isapprox(ẋ1,dynamics_Q(x1),atol=1e-10)
end

Test Summary:           | Pass  Total
attitude dynamics (DCM) |    1      1


Test.DefaultTestSet("attitude dynamics (DCM)", Any[], 1, false, false)

In [45]:
#Implement the full attitude dynamics using a quaternion

function dynamics_q(x)
    
    # TODO: implement attitude dynamics with quaternion 
    ẋ = zeros(length(x))
    
    return ẋ
end

function dynamics_q!(ẋ,x,p,t)
    # make the ODE in-place so it works with OrdinaryDiffEq
    ẋ .= dynamics_q(x)
end

@testset "attitude dynamics (quaternion)" begin
    r1 = normalize([1;2;3])
    θ1 = deg2rad(45)
    q1 = [cos(θ1/2);r1*sin(θ1/2)]
    x1 = [q1;0.2;7;0.3]
    ẋ1 = [-0.7721871929186838;
         -0.9508318305595436;
          3.2489198311984837;
          0.47609421287426346;
         -2.0999999999999996;
          0.06000000000000001;
         -0.46666666666666673]
    @test isapprox(dynamics_q(x1),ẋ1,atol=1e-10)
    
end

Test Summary:                  | Pass  Total
attitude dynamics (quaternion) |    1      1


Test.DefaultTestSet("attitude dynamics (quaternion)", Any[], 1, false, false)

In [46]:
#Initial Conditions
Q0 = Array(I(3))
q0 = [1.0; 0; 0; 0]
Random.seed!(1234)
ω0 = [0; 2*pi; 0] + 1e-2*randn(3) #small perturbation from intermediate-axis spin

x0Q = [Q0[:]; ω0]
x0q = [q0; ω0];

In [47]:
#Set up and solve with OrdinaryDiffEq.jl with default settings
Tf = 60.0
tspan = (0.0, Tf)

prob_q = ODEProblem(dynamics_q!, x0q, tspan)
prob_Q = ODEProblem(dynamics_Q!, x0Q, tspan)

sol_q = solve(prob_q,Tsit5());
sol_Q = solve(prob_Q,Tsit5());


In [48]:
@testset "integration of attitude dynamics" begin
    q1 = sol_q(0)[1:4]
    qf = sol_q(Tf)[1:4]
    norm(q1)
    @test isapprox(1.0,norm(q1),atol=1e-10)
    @test 1 <= norm(qf) <= 1.1
    
    h1 = norm(J*sol_q(0)[5:7])
    hf = norm(J*sol_q(Tf)[5:7])
    @test 12.53 <= h1 <= 12.6
    @test 12.53 <= hf <= 13.0
    
    Q1 = reshape(sol_Q(0)[1:9],3,3)
    Qf = reshape(sol_Q(Tf)[1:9],3,3)
    
    @test norm(Q1'*Q1 - I) < 1e-10
    @test 0 < norm(Qf'*Qf - I) < 0.2
    
    h1 = norm(J*sol_Q(0)[10:12])
    hf = norm(J*sol_Q(Tf)[10:12])
    @test 12.53 <= h1 <= 12.6
    @test 12.53 <= hf <= 13.0
    
end

Test Summary:                    | Pass  Total
integration of attitude dynamics |    8      8


Test.DefaultTestSet("integration of attitude dynamics", Any[], 8, false, false)

In [15]:
plot(sol_q, vars=(5,6,7))

In [16]:
using MeshCat
using CoordinateTransformations
using Rotations
using GeometryBasics: HyperRectangle, Vec, Point, Mesh
using Colors: RGBA, RGB

# Create a new visualizer instance
vis = Visualizer()
render(vis)


┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /Users/kevintracy/.julia/packages/MeshCat/RLYij/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8701))

In [17]:
# meshcat stuff
# delete!(vis)
t_vec = 1:0.01:60
q_hist = sol_q(t_vec).u
box = HyperRectangle(Vec(-0.1, -1, -0.1), Vec(0.2, 2, 0.2))
box2 = HyperRectangle(Vec(-0.1 - 1, -1, -0.1), Vec(2, 0.2, 0.2))
green_material = MeshPhongMaterial(color=RGBA(1, 1, 0.3, 0.5))
setobject!(vis["group1"]["greenbox"], box, green_material)
blue_material = MeshPhongMaterial(color=RGBA(0, 0, 1, 0.5))
setobject!(vis["group1"]["bluebox"], box2, blue_material)
settransform!(vis["group1"], Translation(0, 0, 0))
# animation loop
for i = 1:length(t_vec)
    settransform!(vis, LinearMap(UnitQuaternion(q_hist[i][1:4])  ))
    sleep(0.005)
end

In [18]:
#Plot the error in the quaternion norm vs. time
qnorm = zeros(length(sol_q.t))
for k = 1:length(sol_q.t)
    
    # TODO: calculate the norm of each quaternion 
    qnorm[k] = 100.0 
    
end

plot(sol_q.t,qnorm,xlabel="Time (sec)",ylabel="||q||",label="",lw=2)

In [19]:
#Plot the orthogonality error vs. time for the rotation matrix
Qerr = zeros(length(sol_Q.t))
for k = 1:length(sol_Q.t)
    Qk = reshape(sol_Q.u[k][1:9],3,3)
    
    # TODO: calculate the orthogonality error of the rotation matrix 
    Qerr[k] = 100.0 

end

plot(sol_Q.t,Qerr,xlabel="Time (sec)",ylabel="Orthogonality Error",label="",lw=2)

# Part (c): Correcting attitude integration errors (20 pts)
If you're writing your own integrator, you can easily re-normalize the quaternion at each step. Another option that can work if you're using an off-the-shelf ODE solver is to add proportional feedback on the quaternion norm error to the dynamics. This is an example of an idea called Baumgarte stabilization that can be used to make Runge-Kutta methods better respect known constraints and conserved quantities. Implement it and test some feedback gains between 1 and 20 before choosing a value that you find works best.

In [20]:
function dynamics_q_stabilized(x)
    # TODO: implement stabilized quaternion attitude dynamics 
    # x = [q; ω]
    q = x[1:4]
    ω = x[5:7]
    
    ẋ = zeros(length(x))
    
    return ẋ
end

function dynamics_q_stabilized!(ẋ,x,p,t)
    # make the ODE in-place so it works with OrdinaryDiffEq
    ẋ .= dynamics_q_stabilized(x)
end

dynamics_q_stabilized! (generic function with 1 method)

In [21]:
prob_q2 = ODEProblem(dynamics_q_stabilized!, x0q, tspan)
sol_q2 = solve(prob_q2,Tsit5());

In [22]:
#Plot norm(q) with and without stabilization
qnorm2 = zeros(length(sol_q2.t))
for k = 1:length(sol_q2.t)
    
    # TODO: calculate quaternion norm 
    qnorm2[k] = 100.0 
    
end

plot(sol_q.t,qnorm,xlabel="Time (sec)",ylabel="||q||",label="No Stabilization",lw=2)
plot!(sol_q2.t,qnorm2,label="Stabilization",lw=2)

In [49]:
@testset "stabilized attitude dynamics (quaternion)" begin
    r1 = normalize([1;2;3])
    θ1 = deg2rad(45)
    q1 = [cos(θ1/2);r1*sin(θ1/2)]
    ω1 = [0.2;7;0.3]
    ẋ1 = dynamics_q([q1;ω1])
    ẋ2 = dynamics_q([q1*1.1;ω1])    
    @test isapprox(ẋ1[5:7],ẋ2[5:7],atol=1e-12)
    @test norm(ẋ1 - dynamics_q_stabilized([q1*1.000000001;ω1]))<norm(ẋ1 - dynamics_q_stabilized([q1*1.01;ω1]))
    @test qnorm2[end]<qnorm[end]
end

Test Summary:                             | Pass  Total
stabilized attitude dynamics (quaternion) |    3      3


Test.DefaultTestSet("stabilized attitude dynamics (quaternion)", Any[], 3, false, false)

Unfortunately, the stabilization idea doesn't work as well with rotation matrices since the constraints are much more complicated. Instead, we can implement a Runge-Kutta method that automatically preserves orthogonality of rotation matrices (or normalization of quaternions) by using the exponential map and group multiplication operations. Let's do this for the rotation matrix case first by modifying the explicit midpoint method. This is a simple example of what's known as a Runge-Kutta-Munthe-Kaas method. It is possible to construct an RK-MK method that stays on a Lie group from any standard RK method.

In [25]:
function lie_midpoint_step_Q(xk,h)
    
    Qk = reshape(xk[1:9],3,3)
    ωk = xk[10:12]
    
    # TODO: implement the lie midpoint step 
    ẋ = zeros(length(xk))
    
    return ẋ
end

@testset "lie midpoint (DCM)" begin 
    Q = exp(hat([1,2,3.0]))
    ω = [.2; 7; -.3]
    h = 0.1 
    ẋ = [-0.6027631402683421, -0.7420787949334725, 0.2932501642692329,
          0.6865916167045675, -0.2951187547149963, 0.6644523101671616,
         -0.40653234632050517, 0.6018504654194051, 0.6873917868796442,
          0.42623633333333333, 6.990138333333333, -0.37113616666666666]
    @test isapprox(lie_midpoint_step_Q([vec(Q);ω],h),ẋ,atol=1e-10)
end

Test Summary:      | Pass  Total
lie midpoint (DCM) |    1      1


Test.DefaultTestSet("lie midpoint (DCM)", Any[], 1, false, false)

In [26]:
#Now let's simulate the same example and plot the orthogonality error again

h = 0.1
N = N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1))

xtrajQ = zeros(12,N)
xtrajQ[:,1] .= x0Q

for k = 1:(N-1)
    xtrajQ[:,k+1] .= lie_midpoint_step_Q(xtrajQ[:,k], h)
end

Qerr2 = zeros(N)
for k = 1:N
    Qk = reshape(xtrajQ[1:9,k],3,3)
    
    # TODO: calculate the orthogonality error of Q
    Qerr2[k] = 100.0 
    
end

plot(sol_Q.t,Qerr,xlabel="Time (sec)",ylabel="Orthogonality Error",label="RK",lw=2)
plot!(thist,Qerr2,label="Lie Group",lw=2)

In [53]:
@testset "Lie group DCM" begin 
    @test isapprox(maximum(Qerr2),0.0,atol = 1e-8)
    @test isapprox(minimum(Qerr2),0.0,atol = 1e-8)
end

Test Summary: | Pass  Total
Lie group DCM |    2      2


Test.DefaultTestSet("Lie group DCM", Any[], 2, false, false)

Now let's implement the same Lie group idea but use quaternions instead of rotation matrices. To do this, we'll also need to implement the quaternion exponential map. See if you can figure out a way to implement it without introducing divide-by-zero issues when the angular velocity goes to zero (hint: look into the sinc function).

In [54]:
function Expq(ϕ)
    
    # TODO: implement the quaternion exponential map ϕ → q 
    q = zeros(4)
    
    return q
end

function lie_midpoint_step_q(xk,h)
    
    qk = xk[1:4]
    ωk = xk[5:7]
    
    # TODO: implement lie midpoint step 
    xn = zeros(length(xk))
    
    return xn
end


@testset "Expq" begin 
    q = Expq(zeros(3))
    @test isapprox(norm(q),1,atol = 1e-10)
    @test isapprox(q,[1;0;0;0],atol=1e-10)
    for i = 1:10 
        q = Expq(randn(3))
        @test isapprox(norm(q),1,atol = 1e-10)
    end
    
    # test if the rotation is correct
    Random.seed!(1234)
    ϕ = randn(3)
    q = Expq(ϕ)
    Q = exp(hat(ϕ))
    x = randn(3)
    x2 = Q*x
    x3 = H'*L(q)*R(q)'*H*x
    @test isapprox(x2,x3,atol = 1e-10)
end

@testset "lie midpoint (quaternion)" begin 
    q = Expq([1,2,3.0])
    ω = [.2; 7; -.3]
    h = 0.1 
    ẋ = [-0.4442718458039811, -0.035227218052985225, 0.39378058569262325,
          0.8039393139197423, 0.42623633333333333, 6.990138333333333,
         -0.37113616666666666]
    @test isapprox(lie_midpoint_step_q([q;ω],h),ẋ,atol=1e-10)
end

Test Summary: | Pass  Total
Expq          |   13     13
Test Summary:             | Pass  Total
lie midpoint (quaternion) |    1      1


Test.DefaultTestSet("lie midpoint (quaternion)", Any[], 1, false, false)

In [55]:
#Now let's simulate and plot the quaternion norm again

xtrajq = zeros(7,N)
xtrajq[:,1] .= x0q

for k = 1:(N-1)
    xtrajq[:,k+1] .= lie_midpoint_step_q(xtrajq[:,k], h)
end

qnorm3 = zeros(N)
for k = 1:N
    # TODO: calculate quaternion norm 
    qnorm3[k] = 100.0 
    
end

plot(sol_q.t,qnorm,label="No Stabilization",xlabel="Time (sec)",ylabel="||q||",lw=2)
plot!(sol_q2.t,qnorm2,label="Stabilization",lw=2)
plot!(thist,qnorm3,label="Lie Group",lw=2)

In [56]:
using Statistics
# test that lie group is perfect 
@testset "lie group stabilization" begin 
    @test mean(qnorm3) < mean(qnorm)
    @test mean(qnorm2) < mean(qnorm)
    @test abs(1-mean(qnorm3)) < 1e-6
    @test abs(1-mean(qnorm2)) < 1e-4
end

Test Summary:           | Pass  Total
lie group stabilization |    4      4


Test.DefaultTestSet("lie group stabilization", Any[], 4, false, false)